# Implementational remarks

In this section we discuss implementational topics based on our own experience with boundary element software development.

## Parallelisation and vectorisation

The multithreaded implementation of dense boundary element methods is straight forward. The interaction between any two triangles is independent of each other.

The main difficulty appears upon summing local triangle contributions into the global BEM matrix. Consider for example the space of continuous, piecewise linear, basis functions. Each global dof consists of contributions from multiple triangles. In order to avoid writing into the same global dof at the same time from different threads one needs to either declare the global dof as atomic, or if this is not possible in the chosen language, use techniques such as graph coloring to ensure that at any given time we only process triangle pairs that do not sum into the same global dof.

The idea of map coloring is very simple. We declare two triangles as neighbours if they sum into the global dof. We can then use simple greedy colouring techniques to assign a colour to each triangle so that neighbours always have a different colour. We can then process in parallel one colour after another.

In addition to simple multithreading, SIMD vectorisation is extremely important to achieve performance. All modern CPUs support vector registers for floating point operations. This means that each core can execute several floating point operations simultaneously.

On Intel and AMD CPUs the main standard is AVX2, which supports up to four parallel double precision or up to 8 parallel single precision operations.

The way we implement vectorisation in Bempp is that we always process one test triangle simultaneously with 4/8 trial triangles. The caveat here though is what to do with adjacent triangles. Vectorisation only works if there is no divergence between the pipelines.

Here, we use a small trick. We ignore the check whether some of the test/trial pairs are adjacent. We just process everything in a batch with the far-field rule. We then afterwards check if some of the interactions were from adjacent triangles and do not sum those into the global matrix. The singular interactions are then processed independently from the nonsingular interactions.

## CPU vs GPU assembly

This is a frequently coming up question. In principle, GPUs are ideally suited for fast assembly of integral operators. We have extremely high throughput of data on GPUs and each individual interaction of a test/trial triangle pair can be easily handled as a GPU thread.

The problem is memory latency. If we want to transfer an assembled matrix back to the CPU the transfer time can easily invalidate all performance gains from running the assembly on the GPU. Hence, if we want to use GPUs we need to keep the data on the GPU. But then we run into the problem of GPU memory being typically much smaller than CPU memory. So we can only keep the data for relatively small problems on the GPU.

A solution to this issue is on-the-fly assembly on the GPU as part of the matrix-vector product. This means that as part of the matvec $Ax$ we assemble the matrix elements of $A$ on the fly while passing through the matvec and immediately discard computed matrix elements. This keeps the required memory small and for simple kernels such as Laplace the assembly is so fast that the overhead is relatively small.

We have done successful experiments with this strategy with Maxwell problems on up to 200k unknowns while still achieving good performance for each matvec. Indeed, such an approach can outperform FMM and other fast evaluation methods for numbers of dofs in the low hundreds of thousands. Nevertheless, eventually the quadratic complexity takes over and fast solver methods become necessary.

## Programming languages

We have used for BEM development C++, Python, and now Rust. The programming language in principle does not matter. For interactive applications a mixture of low-level code for performant assembly and high-level interfaces is recommended. Also, the environment should have good support for SIMD execution as discussed above.

C++ is an excellent choice in general for modern low-level HPC development due to its great support for heterogeneous computing frameworks (CUDA, SYCL, etc.). Python needs to be combined with a low-level computational part. We have some experience with Numba for Just-In-Time compiled fast code in Python. Numba is good for simple array algorithms but can run into problems for modern structures.

Julia can produce very fast low-level code. Nevertheless, fast code development in Julia is not much different from code development in other low-level languages.

Rust is a very young language but has a excellent tooling support, much better dependency management than C++, and a growing HPC ecosystem. It is definitely more than capable for complex HPC applications and achieves excellent performance. Its only caveat compared to C++ is that GPU support on Rust is still in its infancy.